In [1]:
import pandas as pd
import numpy as np


In [2]:
# чтобы добраьтся до папки helpers, в которой лежит metrics_f1.py
from os import getcwd
import sys
sys.path.append('\\'.join(getcwd().split('\\')[:-1]))
from helpers.metrics_f1 import calc_f1_score

In [3]:
path_train = r"../data/train_1"

In [4]:
# данные по дислокации
dislok = pd.read_parquet(path_train + '/dislok_wagons.parquet').convert_dtypes()
# данные по текущим ремонтам
pr_rem = pd.read_parquet(path_train + '/pr_rems.parquet').convert_dtypes()
# список вагонов с остаточным пробегом на момент прогноза
wag_prob = pd.read_parquet(path_train + '/wagons_probeg_ownersip.parquet').convert_dtypes()
# параметры вагона
wag_param = pd.read_parquet(path_train + '/wag_params.parquet').convert_dtypes()
# таргет по прогноза выбытия вагонов в ПР на месяц и на 10 дней
target = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()
# текущие ремонты вагонов
tr_rem = pd.read_parquet(path_train + '/tr_rems.parquet').convert_dtypes()
# --------------------------------- y  ----------------------------------------

In [5]:
wag_param = wag_param.drop_duplicates(subset='wagnum', keep='last')# у вагонов могут меняться параметры, поэтмоу номер дублируется. В данной модели это фактор не учитывается

In [6]:
month_to_predict = pd.to_datetime('2022-12-01')

In [7]:
target.month = pd.to_datetime(target.month)
target = target[target.month == month_to_predict][['wagnum','target_month','target_day']]

In [8]:
target.target_month.sum(), target.target_day.sum()

(1584, 570)

# Наивная модель

Наивная модель будет построена на правилах с использованием минимального набора данных, без применения Ml.

Реальный процесс выглядит следующим образом - в начале месяца берется срез по парку по всем вагонам, за ремонт которых несёт ответственность ПГК. Для выбранных вагонов требуется установить, какие из них будут отремонтированы в текущем месяце. Данная информация помогает планировать нагрузку на вагоно-ремонтное предприятие(ВРП). Вторая модель определяет критичные вагоны, которые будут отправлены в ремонт в первую очередь( в ближайшие 10 дней). Это помогает фокусировать внимание диспетчеров.

Основными критериями по которым вагон отправляется в плановый ремонт - является его остаточный пробег и срок до планового ремонта.
В регламентах РЖД используется следующее правило - если ресурс по пробегу не превышает 500 км и/или плановый ремонт должен наступить через 15 дней(или меньше), то вагон может ехать только на ВРП.
Из этого регламента вытекают две особенности:
1. Диспетчер старается отправить вагон раньше положенных значений. Это позволяет выбрать предприятия, на которых ремонтироваться дешевле, а не ближайшее.
2. Компания-оператор может выбирать какому из нормативов нужно следовать - ремонтировать вагон по сроку, или по пробегу, или по обоим критериям сразу. Поэтому встречаются вагоны, у которых пробег может не отслеживаться.

Вагон может быть отправлен в плановый ремонт и раньше положенного. На это может влиять, например, история грузовых операций и количество текущих(мелких) ремонтов. Основная цель участников в данной задаче - найти закономерности и оценить значимые признаки, по которым вагон выбывает в плановый ремонт.

## Подготовка данных

In [9]:
# оставим только данные по остаточному пробегу для каждого номерав вагона
wag_prob = wag_prob[(wag_prob.repdate == month_to_predict) | (wag_prob.repdate == wag_prob.repdate.min())]
wag_prob.head()

,repdate,wagnum,ost_prob,manage_type,rod_id,reestr_state,ownership_type,month
0,2022-08-01,33361,7541,0,1,1,0,8
122,2022-12-01,33361,159916,0,1,1,0,12
273,2022-08-01,33364,37103,0,1,1,0,8
395,2022-12-01,33364,4268,0,1,1,0,12
546,2022-08-01,33366,10242,0,1,1,0,8


In [10]:
# оценим среднесуточный пробег из данных по пробегу вагона, на тот случай, если данных по нормативу нет
wag_prob_ =wag_prob.groupby('wagnum', as_index = False).agg({'repdate':['max', 'min'] , 'ost_prob': ['max','min']},)#.droplevel(1)
wag_prob_.columns = [head+'_' + name
                     if head!='wagnum'
                     else head
                     for head, name in wag_prob_.columns ]

wag_prob_['diff_days'] = wag_prob_.repdate_max - wag_prob_.repdate_min
wag_prob_['mean_run'] = (wag_prob_.ost_prob_max - wag_prob_.ost_prob_min )/ wag_prob_.diff_days.dt.days
wag_prob = wag_prob[wag_prob.repdate == wag_prob.repdate.max()][['wagnum','ost_prob']]
wag_prob = wag_prob.merge(wag_prob_[['wagnum','mean_run']])
wag_prob.head()

,wagnum,ost_prob,mean_run
0,33361,159916,1248.975410
1,33364,4268,269.139344
2,33366,1507,71.598361
3,33358,30223,95.114754
4,33349,153839,1214.073770


In [11]:
# для каждого вагона оставим только информацию по сроку службы и нормативу суточного пробега между ПР
wag_param = wag_param[['wagnum','srok_sl','cnsi_probeg_dr','cnsi_probeg_kr']]
wag_param.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr
3218,26318,2022-04-27,160,160
19128,28344,2024-12-24,110,160
21526,8099,2027-10-01,110,160
32353,33350,2047-02-05,250,500
81,5308,2027-09-28,110,160


In [12]:
# добавим признак, что вагон был в ПР в предыдущем месяце. Скорее всего, если вагон был в ПР недавно, то повторно он не поедет
pr_rem['was_repair_in_prev_month'] = 1
pr_rem = pr_rem[['wagnum','was_repair_in_prev_month']]
pr_rem = pr_rem.drop_duplicates(subset='wagnum') #некоторые вагоны все же ремонтируются больше 1 раза, поэтому нужен сбросить дубли
pr_rem.head()

,wagnum,was_repair_in_prev_month
0,15000,1
2,25485,1
3,25944,1
4,24080,1
5,28612,1


In [13]:
# посчитаем сколько текущих ремонтов было за прошедший период
tr_rem = tr_rem.groupby('wagnum', as_index= False).kod_vrab.count()
tr_rem.head()

,wagnum,kod_vrab
0,0,2
1,2,2
2,3,2
3,6,9
4,8,2


In [14]:
# сохраним только дату следующего планового ремонта для вагона
dislok = dislok[['wagnum','date_pl_rem']].drop_duplicates(subset = 'wagnum', keep='last')
dislok.head(100)

,wagnum,date_pl_rem
347,11219,2019-06-27
25426,33350,2022-07-25
216913,8099,2023-04-20
730409,28344,2022-11-30
993683,26318,2023-01-01
...,...,...
2634,12723,2023-06-15
2665,15720,2023-11-09
2696,29998,2024-04-06
2727,21749,2023-05-29


In [15]:
# соберем все данные вместе
wp = target[['wagnum']].merge(wag_param, on ='wagnum', how = 'left')\
             .merge(wag_prob, how = 'left')\
             .merge(pr_rem, how = 'left')\
             .merge(tr_rem, how = 'left')\
             .merge(dislok, how = 'left')

In [16]:
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem
0,33361,2033-03-01,110,160,159916,1248.975410,1.0,3.0,2023-02-17
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23
4,33349,2033-12-04,110,160,153839,1214.073770,1.0,NaN,2023-07-06


In [17]:
# Получим среднесуточный пробег, как среднее от нормативов и реального пробега
wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']] = wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']].fillna(0)
wp['day_run'] = wp.apply(lambda x : [ val  for val in [x.cnsi_probeg_kr, x.cnsi_probeg_dr, x.mean_run] if val != 0], axis = 1 )
wp['day_run']= wp.apply(lambda x : np.mean(x.day_run) if len(x.day_run)> 0 else 0, axis = 1 )
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run
0,33361,2033-03-01,110,160,159916,1248.975410,1.0,3.0,2023-02-17,506.325137
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,179.713115
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,113.866120
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,121.704918
4,33349,2033-12-04,110,160,153839,1214.073770,1.0,NaN,2023-07-06,494.691257


In [18]:
wp['current_date'] = month_to_predict

In [19]:
# определим, сколько дней осталось до истечения срока службы
wp['date_diff_srk_sl'] = wp['srok_sl']- wp['current_date']
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run,current_date,date_diff_srk_sl
0,33361,2033-03-01,110,160,159916,1248.975410,1.0,3.0,2023-02-17,506.325137,2022-12-01,3743 days
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,179.713115,2022-12-01,3054 days
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,113.866120,2022-12-01,3338 days
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,121.704918,2022-12-01,3652 days
4,33349,2033-12-04,110,160,153839,1214.073770,1.0,NaN,2023-07-06,494.691257,2022-12-01,4021 days


In [20]:
# определим, сколько дней осталось до ближайшего ПР
wp['date_diff_pl_rem'] = wp['date_pl_rem']- wp['current_date']
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run,current_date,date_diff_srk_sl,date_diff_pl_rem
0,33361,2033-03-01,110,160,159916,1248.975410,1.0,3.0,2023-02-17,506.325137,2022-12-01,3743 days,78 days
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,179.713115,2022-12-01,3054 days,306 days
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,113.866120,2022-12-01,3338 days,123 days
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,121.704918,2022-12-01,3652 days,449 days
4,33349,2033-12-04,110,160,153839,1214.073770,1.0,NaN,2023-07-06,494.691257,2022-12-01,4021 days,217 days


In [21]:
# определим, какой остаточный ресурс будет на момент окончания месяца
wp['prob_end_month'] = wp['ost_prob'] - wp['day_run']* 30

In [22]:
wp['target_month'] = 0
wp['target_day'] = wp['target_month']

In [23]:
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run,current_date,date_diff_srk_sl,date_diff_pl_rem,prob_end_month,target_month,target_day
0,33361,2033-03-01,110,160,159916,1248.975410,1.0,3.0,2023-02-17,506.325137,2022-12-01,3743 days,78 days,144726.245902,0,0
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,179.713115,2022-12-01,3054 days,306 days,-1123.393443,0,0
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,113.866120,2022-12-01,3338 days,123 days,-1908.983607,0,0
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,121.704918,2022-12-01,3652 days,449 days,26571.852459,0,0
4,33349,2033-12-04,110,160,153839,1214.073770,1.0,NaN,2023-07-06,494.691257,2022-12-01,4021 days,217 days,138998.262295,0,0


In [24]:
# Мои расчеты

# показания замеров
kti_df = pd.read_parquet(path_train + './kti_izm.parquet').convert_dtypes()

# Преобразование колонки 'timestamp' в читаемый формат
kti_df['moment'] = kti_df['operation_date_dttm'].apply(lambda x: pd.to_datetime(x , utc=True))

# Примените фильтр по периоду
start_date = '2023-01-01'
end_date = '2023-02-28'
df_filtered = kti_df[(kti_df['moment'] >= start_date) & (kti_df['moment'] <= end_date)]

# Сгруппируйте данные по 'wagnum' и получите средние значения для каждой группы
df_grouped = df_filtered.groupby('wagnum').agg({
    'mileage_all': 'mean',
    'axl1_l_w_flange': 'mean',
    'axl1_r_w_flange': 'mean',
    'axl2_l_w_flange': 'mean',
    'axl2_r_w_flange': 'mean',
    'axl3_l_w_flange': 'mean',
    'axl3_r_w_flange': 'mean',
    'axl4_l_w_flange': 'mean',
    'axl4_r_w_flange': 'mean',
    'axl1_l_w_rim': 'mean',
    'axl1_r_w_rim': 'mean',
    'axl2_l_w_rim': 'mean',
    'axl2_r_w_rim': 'mean',
    'axl3_l_w_rim': 'mean',
    'axl3_r_w_rim': 'mean',
    'axl4_l_w_rim': 'mean',
    'axl4_r_w_rim': 'mean'
}).reset_index()
# 
# # Выведите результат
# print(df_grouped)

# date_string = "01.01.2023 00:00:00"
# timestamp = pd.to_datetime(date_string).timestamp()
# 
# print(timestamp)
# kti_df = df_grouped.drop('operation_date_dttm', axis=1)
# kti_df.head()
# kti_df.tail()

# 34 - 24 flange
# 80 - 25 rim

# теперь еще сделаем признаком что если меньше 28
# и меньше 32  
merged_df = pd.merge(wp, df_grouped, on='wagnum', how='left')  # 'inner' - внутреннее объединение


merged_df.head(100)

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,2033-03-01,110,160,159916,1248.975410,1.0,3.0,2023-02-17,506.325137,...,27.285714,30.442857,60.442857,60.40,64.885714,63.871429,57.757143,56.657143,56.171429,55.185714
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,179.713115,...,32.675000,31.150000,75.925000,76.50,77.325000,74.300000,77.125000,75.275000,76.425000,75.525000
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,113.866120,...,28.780000,29.020000,67.860000,68.06,67.320000,67.560000,65.500000,65.000000,66.720000,66.900000
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,121.704918,...,29.700000,28.533333,52.433333,53.10,44.666667,45.083333,51.233333,53.083333,57.783333,56.860000
4,33349,2033-12-04,110,160,153839,1214.073770,1.0,NaN,2023-07-06,494.691257,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8557,2026-04-10,110,160,100814,87.713115,NaN,4.0,2024-02-18,119.237705,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,9058,2030-09-20,110,160,37663,144.745902,NaN,2.0,2023-09-16,138.248634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,9692,2029-06-17,110,160,113852,104.778689,NaN,2.0,2024-07-20,124.926230,...,29.800000,28.400000,65.775000,66.85,54.925000,54.675000,64.850000,65.550000,65.200000,65.925000
98,18338,2042-02-21,110,160,93132,157.401639,NaN,NaN,2024-08-25,142.467213,...,28.200000,30.650000,75.300000,76.35,76.800000,78.950000,79.650000,78.850000,72.600000,73.050000


In [25]:
# вагон выбывает в ПР в следующем месяце, если:
# остаточный пробег < 5 000 км
# срок службы < 500 лней
# до следующего  ПР < 40 дней
# ,число текущих ремонтов > 5
merged_df.loc[(merged_df.prob_end_month <= 4500) \
       | (merged_df.date_diff_srk_sl < pd.to_timedelta(35))\
       | (merged_df.date_diff_pl_rem < pd.to_timedelta(12)) \
       | (merged_df.axl1_l_w_flange < 24) \
       | (merged_df.axl1_r_w_flange < 24) \
       | (merged_df.axl2_l_w_flange < 24) \
       | (merged_df.axl2_r_w_flange < 24) \
       | (merged_df.axl3_l_w_flange < 24) \
       | (merged_df.axl3_r_w_flange < 24) \
       | (merged_df.axl4_l_w_flange < 24) \
       | (merged_df.axl4_r_w_flange < 24) \
       | (merged_df.axl1_l_w_rim < 24) \
       | (merged_df.axl1_r_w_rim < 24) \
       | (merged_df.axl2_l_w_rim < 24) \
       | (merged_df.axl2_r_w_rim < 24) \
       | (merged_df.axl3_l_w_rim < 24) \
       | (merged_df.axl3_r_w_rim < 24) \
       | (merged_df.axl4_l_w_rim < 24) \
       | (merged_df.axl4_r_w_rim < 24) \
       | (merged_df.kod_vrab > 20),'target_month'] = 1

merged_df.loc[(merged_df.prob_end_month <= 3000) \
              | (merged_df.date_diff_srk_sl < pd.to_timedelta(15)) \
              | (merged_df.date_diff_pl_rem < pd.to_timedelta(5)) \
              | (merged_df.axl1_l_w_flange < 23) \
              | (merged_df.axl1_r_w_flange < 23) \
              | (merged_df.axl2_l_w_flange < 23) \
              | (merged_df.axl2_r_w_flange < 23) \
              | (merged_df.axl3_l_w_flange < 23) \
              | (merged_df.axl3_r_w_flange < 23) \
              | (merged_df.axl4_l_w_flange < 23) \
              | (merged_df.axl4_r_w_flange < 23) \
              | (merged_df.axl1_l_w_rim < 23) \
              | (merged_df.axl1_r_w_rim < 23) \
              | (merged_df.axl2_l_w_rim < 23) \
              | (merged_df.axl2_r_w_rim < 23) \
              | (merged_df.axl3_l_w_rim < 23) \
              | (merged_df.axl3_r_w_rim < 23) \
              | (merged_df.axl4_l_w_rim < 23) \
              | (merged_df.axl4_r_w_rim < 23) \
              | (merged_df.kod_vrab > 10),'target_day'] = 1

wp = merged_df

In [26]:
# wp['target_day'] = wp['target_month']

In [27]:
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,2033-03-01,110,160,159916,1248.975410,1.0,3.0,2023-02-17,506.325137,...,27.285714,30.442857,60.442857,60.40,64.885714,63.871429,57.757143,56.657143,56.171429,55.185714
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,179.713115,...,32.675000,31.150000,75.925000,76.50,77.325000,74.300000,77.125000,75.275000,76.425000,75.525000
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,113.866120,...,28.780000,29.020000,67.860000,68.06,67.320000,67.560000,65.500000,65.000000,66.720000,66.900000
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,121.704918,...,29.700000,28.533333,52.433333,53.10,44.666667,45.083333,51.233333,53.083333,57.783333,56.860000
4,33349,2033-12-04,110,160,153839,1214.073770,1.0,NaN,2023-07-06,494.691257,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
pred_target = target[['wagnum']].merge(wp[['wagnum','target_month','target_day']],how = 'left')
pred_target = pred_target.drop_duplicates(subset = 'wagnum')

In [29]:
# Проверим соотношение отмеченных вагонов с фактическим значением
round(pred_target.target_month.sum() / target.target_month.sum(), 2)

2.39

In [30]:
# сохраним таргет за месяц  для выбранного периода отдельно
target_path = r'../data/train_1/prediction/target_predicton.csv'

In [31]:
pred_target.drop_duplicates(subset = 'wagnum').to_csv(target_path, index=False)

In [32]:
true_target_path = '../data/train_1/prediction/target_predicton_true.csv'

In [33]:
target.drop_duplicates(subset = 'wagnum').to_csv(true_target_path, index=False)

In [34]:
# оценим насколько хорошо удалось предсказать выбытие вагонов  по месяцу и по 10 дням
calc_f1_score(true_target_path, target_path,)

0.21237433346595525